In [1]:
#Importing libraries
!pip install vaderSentiment
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

     |████████████████████████████████| 133kB 3.9MB/s 


In [2]:
#loading set
from google.colab import drive
drive.mount('/content/drive')

import glob

path = r'drive/MyDrive/Sentiment analysis/' # use your path
all_files = glob.glob(path + "/*.CSV")
#print(all_files)
li = []

for filename in all_files:
    df = pd.read_csv(filename, usecols = ['text'], index_col=None, header=0, nrows = 8000,encoding = "ISO-8859-1")
    li.append(df)


df = pd.read_csv("drive/MyDrive/Sentiment analysis/covid19_tweets.csv",usecols = ['text'],encoding = "ISO-8859-1",nrows = 178687)
li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
tweets = pd.DataFrame(frame)
print(tweets)
index = tweets.index
noOfTweet = len(index)

Mounted at /content/drive
['drive/MyDrive/Sentiment analysis/2020-04-16 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-17 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-18 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-19 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-20 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-21 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-22 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-23 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-24 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-25 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-26 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-27 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-28 Coronavirus Tweets.CSV', 'drive/MyDrive/Sentiment analysis/2020-04-29 Coronavirus Tweets

In [4]:
#cleaning the tweets
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, str(input_txt))
  for i in r:
    input_txt = re.sub(i, '', str(input_txt))        
  return input_txt

def clean_tweets(tweet):
  #remove twitter Return handles (RT @xxx:)
  tweet = np.vectorize(remove_pattern)(tweet, "RT @[\w]*:")
    
  #remove twitter handles (@xxx)
  tweet = np.vectorize(remove_pattern)(tweet, "@[\w]*")
    
  #remove URL links (httpxxx)
  tweet = np.vectorize(remove_pattern)(tweet, "https?://[A-Za-z0-9./]*")
    
  #remove special characters, numbers, punctuations (except for #)
  tweet = np.core.defchararray.replace(tweet, "[^a-zA-Z]", " ")
    
  return tweet

df = clean_tweets(tweets)
print(df[1:10])

[['PWDs from Tahanang Walang Hagdan get aid amid #COVID19 lockdown ']
 ['#France à¹\x84à¸\x94à¹\x89à¸£à¸±à¸\x9aà¸\x84à¸§à¸²à¸¡à¸\x97à¸¸à¸\x81à¸\x82à¹\x8cà¸\x97à¸£à¸¡à¸²à¸\x99à¸\x88à¸²à¸\x81 # à¸\x8aà¸¸à¸\x94à¸\x81à¸²à¸£à¹\x81à¸¢à¸\x81à¸\x95à¸±à¸§à¸\x97à¸µà¹\x88à¸\x94à¹\x89à¸\xadà¸¢à¸\x81à¸§à¹\x88à¸²à¸\x82à¸\xadà¸\x87à¸\x88à¸µà¸\x99à¸\x8bà¸¶à¹\x88à¸\x87à¹\x84à¸¡à¹\x88à¹\x80à¸\x9eà¸µà¸¢à¸\x87 à¹\x81à¸\x95à¹\x88à¸ªà¸±à¹\x89à¸\x99 à¹\x81à¸\x95à¹\x88à¸¢à¸±à¸\x87à¸\x9eà¸±à¸\x87\n#COVID #COVID19 #à¹\x80à¸\x88à¸²à¸°à¸\x9bà¸£à¸°à¹\x80à¸\x94à¹\x87à¸\x99 #italia #à¹\x82à¸\x84à¸§à¸´à¸\x9419 #à¹\x80à¸£à¸²à¹\x84à¸¡à¹\x88à¸\x97à¸´à¹\x89à¸\x87à¸\x81à¸±à¸\x99  #italy #à¹\x82à¸\x84à¸§à¸´à¸\x94 #à¸£à¸±à¸\x90à¸\x9aà¸²à¸¥à¹\x80à¸®à¸\x87à¸\x8bà¸§à¸¢ #coronavirus #à¸\x96à¸²à¸¡à¸\x95à¸£à¸\x87à¹\x86à¸\x81à¸±à¸\x9aà¸\x88à¸\xadà¸¡à¸\x82à¸§à¸±à¸\x8d #CoronavirusUSA #à¸\xadà¸¢à¸²à¸\x81à¹\x83à¸«à¹\x89à¹\x80à¸«à¹\x87à¸\x99 #coronathailand']
 ['Target ng Department of Health na magkaroon ng 8,000 #COVID19 test kada a

In [5]:
analyzer = SentimentIntensityAnalyzer()

In [6]:
scores = []
# Declare variables for scores
positive = 0
negative = 0
neutral = 0
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
labels = []

for i in range(df.shape[0]):
  compound = analyzer.polarity_scores(df[i])["compound"]
  pos = analyzer.polarity_scores(df[i])["pos"]
  neu = analyzer.polarity_scores(df[i])["neu"]
  neg = analyzer.polarity_scores(df[i])["neg"]
  if(neg > pos):
    negative_list.append(df[i])
    status = "Negative"
    negative+=1
  elif (pos > neg):
    positive_list.append(df[i])
    status = "Positive"
    positive+=1
  elif (pos == neg):
    neutral_list.append(df[i])
    status = "Neutral"
    neutral+=1
  scores.append({"Compound": compound, "Positive": pos, "Negative": neg, "Neutral": neu, "Status": status})
  labels.append(status)

In [7]:
df1 = pd.DataFrame(df)
Y_train = pd.DataFrame.from_dict(scores)
df1 = df1.join(Y_train)
df1.head(1000)

,0,Compound,Positive,Negative,Neutral,Status
0,"Para complementar la higiene de tus manos, el ...",0.0,0.0,0.0,1.0,Neutral
1,PWDs from Tahanang Walang Hagdan get aid amid ...,0.0,0.0,0.0,1.0,Neutral
2,#France à¹à¸à¹à¸£à¸±à¸à¸à¸§à¸²à¸¡à¸à¸¸à¸...,0.0,0.0,0.0,1.0,Neutral
3,Target ng Department of Health na magkaroon ng...,0.0,0.0,0.0,1.0,Neutral
4,La lanzÃ³ una nueva convocatoria especial par...,0.0,0.0,0.0,1.0,Neutral
...,...,...,...,...,...,...
995,"Marcos AssunÃ§Ã£o, diretor de Assuntos Parlame...",0.0,0.0,0.0,1.0,Neutral
996,Parte epidemiolÃ³gico 15/04. Misiones tiene 9...,0.0,0.0,0.0,1.0,Neutral
997,Pdte. envia saludo de agradecimiento a dueÃ±...,0.0,0.0,0.0,1.0,Neutral
998,A pandemia #COVID19 descortinou o brasileiro i...,0.0,0.0,0.0,1.0,Neutral


In [8]:
def percentage(part,whole):
  return 100 * float(part)/float(whole)

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

In [9]:
#Number of Tweets (Total, Positive, Negative, Neutral)
tweet_list = pd.DataFrame(df1)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number:",len(tweet_list))
print("positive number:",len(positive_list))
print("negative number:", len(negative_list))
print("neutral number:",len(neutral_list))

total number:  298687
positive number:  101217
negative number:  78041
neutral number:  119429


In [10]:
df_final = df1[[0,"Status"]].copy()
#print(df_final)
df_final = df_final[df_final.Status != "Neutral"]
df_final[0] = df_final[0].apply(lambda x: x.lower())
df_final[0] = df_final[0].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(df_final[ df_final['Status'] == 'Positive'].size)
print(df_final[df_final['Status'] == 'Negative'].size)

for idx,row in df1.iterrows():
    row[0] = row[0].replace('rt',' ')
X = df_final[0]
#print(X)


202434
156082


In [12]:
import sklearn.model_selection as model_selection
Y = df_final['Status']
#print(Y)
#print(type(Y))
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.80, test_size=0.20, random_state=0)
"""print(X_train)
print(Y_train)"""
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
#print(type(X_train))
"""x_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(Y_train)
x_test = pd.DataFrame(X_test)"""
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,recall_score,precision_score
from sklearn.naive_bayes import MultinomialNB

cv = CountVectorizer()
x_train = cv.fit_transform(X_train)
y_train = Y_train
x_test = cv.transform(X_test)
y_test = Y_test
#print(x_test)

(143406,) (143406,)
(35852,) (35852,)


In [15]:
nb = MultinomialNB()
nb.fit(x_train,y_train)
nb_predict=nb.predict(x_test)
#print(nb_predict)
nb_report = accuracy_score(y_test,nb_predict)
print('Accuracy:',nb_report)
#print(y_test[1:20])
#print(nb_predict[1:20])

nb_report2 = precision_score(y_test,nb_predict,pos_label='Positive',average='binary')
print('Precision:',nb_report2)



Accuracy: 0.8148778310833427
Precision: 0.7877888488446046


In [16]:
twt1 = ['MFirst phase of #COVAXINVaccine was completed at PGIMS Rohtak now, Second phase of #HumanTrials of indigenously']
twt2 = ['Chief Minister @NitishKumar has ordered immediate antigen testing of the flood victims taking shelter in the governance']
twt3 = ['Help slow the spread of #COVID19 and identify at risk cases sooner by self-reporting your symptoms daily, even if']
twt = cv.transform(twt3)
print(nb.predict(twt))

['Positive']
